## Data

In [213]:
import csv
directory = r'C:\Users\Yang\Desktop\project\data\TBM_data.csv'
data = []
f = open(directory, 'r', encoding='utf-8')
rdr = csv.reader(f)
for line in rdr:
    data.append(line)
f.close()  
data[0][0] = 'Sunny' # 자꾸 다른거 포함됨

In [214]:
train_index = [0,1,2,3,5,7,8,11,12]
test_index = [4,6,9,10,13]

train_data = []
test_data = []

for i in train_index:
    train_data.append(data[i])

for i in test_index:
    test_data.append(data[i])

In [215]:
train_data, test_data

([['Sunny', 'Humidity=All', 'Temperature=All', 'FALSE', 'No'],
  ['Sunny', 'Humidity=All', 'Temperature=All', 'TRUE', 'No'],
  ['Overcast', 'Humidity=All', 'Temperature=All', 'FALSE', 'Yes'],
  ['Rainy', 'Humidity=All', 'Temperature=All', 'FALSE', 'Yes'],
  ['Rainy', 'Humidity=All', 'Temperature=All', 'TRUE', 'No'],
  ['Sunny', 'Humidity=All', 'Temperature=All', 'FALSE', 'No'],
  ['Sunny', 'Humidity=All', 'Temperature=All', 'FALSE', 'Yes'],
  ['Overcast', 'Humidity=All', 'Temperature=All', 'TRUE', 'Yes'],
  ['Overcast', 'Humidity=All', 'Temperature=All', 'FALSE', 'Yes']],
 [['Rainy', 'Humidity=All', 'Temperature=All', 'FALSE', 'Yes'],
  ['Overcast', 'Humidity=All', 'Temperature=All', 'TRUE', 'Yes'],
  ['Rainy', 'Humidity=All', 'Temperature=All', 'FALSE', 'Yes'],
  ['Sunny', 'Humidity=All', 'Temperature=All', 'TRUE', 'Yes'],
  ['Rainy', 'Humidity=All', 'Temperature=All', 'TRUE', 'No']])

## Apriori

In [216]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [217]:
min_support = 0.01
min_confidence = 0.1
class_label = ['Yes', 'No']
#class_index = range(0,len(class_label))

In [241]:
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_) #위에서 나온걸 보기 좋게 데이터프레임으로 변경

frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True) # 여러 아이템이 frequent한 것

itemset = [list(x) for x in frequent_itemsets['itemsets']]
frequent_itemset = [] # 단일 아이템이 frequent한 것
for item in itemset:
    if len(item) is 1 and item[0] != class_label[0] and item[0] != class_label[1]:
        frequent_itemset.append(item[0])

all_ar = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence) # 모든 rule들

In [245]:
all_ar.head(100)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Humidity=All),(FALSE),1.000000,0.571429,0.571429,0.571429,1.000000,0.000000,1.000000
1,(FALSE),(Humidity=All),0.571429,1.000000,0.571429,1.000000,1.000000,0.000000,inf
2,(FALSE),(No),0.571429,0.357143,0.142857,0.250000,0.700000,-0.061224,0.857143
3,(No),(FALSE),0.357143,0.571429,0.142857,0.400000,0.700000,-0.061224,0.714286
4,(Overcast),(FALSE),0.285714,0.571429,0.142857,0.500000,0.875000,-0.020408,0.857143
...,...,...,...,...,...,...,...,...,...
95,(FALSE),"(Humidity=All, Yes)",0.571429,0.642857,0.428571,0.750000,1.166667,0.061224,1.428571
96,"(FALSE, Sunny)",(No),0.214286,0.357143,0.142857,0.666667,1.866667,0.066327,1.928571
97,"(FALSE, No)",(Sunny),0.142857,0.357143,0.142857,1.000000,2.800000,0.091837,inf
98,"(Sunny, No)",(FALSE),0.214286,0.571429,0.142857,0.666667,1.166667,0.020408,1.285714


In [218]:
# Car_set을 얻음
def car_set(data, min_support, min_confidence, class_label):
    te = TransactionEncoder()
    te_ary = te.fit(data).transform(data)
    df = pd.DataFrame(te_ary, columns=te.columns_) #위에서 나온걸 보기 좋게 데이터프레임으로 변경
    
    frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True) # 여러 아이템이 frequent한 것

    itemset = [list(x) for x in frequent_itemsets['itemsets']]
    frequent_itemset = [] # 단일 아이템이 frequent한 것
    for item in itemset:
        if len(item) is 1 and item[0] != class_label[0] and item[0] != class_label[1]:
            frequent_itemset.append(item[0])
            
    all_ar = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence) # 모든 rule들
    confidence = list(all_ar['confidence']) # confidence값 저장
    all_ar = all_ar[all_ar['antecedent support'] > 0.01]
    all_ar = all_ar[(all_ar['consequents'] == frozenset({'Yes'})) | (all_ar['consequents'] == frozenset({'No'}))] # rule에서 결과가 class인 것만 불러옴 
    
    ant = [list(x) for x in all_ar['antecedents']] # rule에서 원인
    
    # rule에서 원인이 frequent itemset의 부분집합일 때의 index 저장
    index = []
    count = 0
    for i in ant:
        state = True
        for j in i:
            if j not in frequent_itemset:
                state = False
        if state:
            index.append(count)
        count=count+1
        
    all_ar_list = all_ar.values.tolist() # df -> list
    
    # car-set을 찾음
    car = []
    car_ant = []
    car_con = []
    car_confidence = []
    for i in index:
        car_ant.append(list(all_ar_list[i][0]))
        car_con.append(list(all_ar_list[i][1]))
        car_confidence.append(confidence[i])
    car.append(car_ant)
    car.append(car_con)
    
    return frequent_itemset, car, car_confidence

## Split

In [219]:
# Car_set들을 split함
def split_rule(car, car_confidence):
    label = list(set([i[0] for i in car[1]])) # class label값들 불러옴.
    
    # 각 class에 대한 index를 얻음
    split_index = [] 
    for j in label:
        class_index = []
        index = 0
        for i in car[1]:
            if i[0] == j:
                class_index.append(index)
            index+=1
        split_index.append(class_index)
    
    # 해당하는 class로 분할
    split_car = []
    split_confidence = []
    for i in split_index:
        car_set_ex = []
        confidence_ex = []
        for j in i:
            car_set_ex.append(car[0][j])
            confidence_ex.append(car_confidence[j])
        split_car.append(car_set_ex)
        split_confidence.append(confidence_ex)
    
    # 각 클래스 명과 해당하는 class의 car들에 대응되게 dictionary 형태로 만듬
    rule_class = {}
    for i in range(len(label)):
        rule_class[label[i]] = split_car[i]
    
    return rule_class, split_confidence, label

## Associative classification based on Transferable Belief Model

In [239]:
def normalize_confidence(split_confidence, label):
    split_confidence_np = np.array(split_confidence)
    normalize_confidence = []
    for i in range(len(label)):
        normalize = list((split_confidence_np[i]/np.sum(split_confidence_np[i])))
        normalize_confidence.append(normalize)
        print(np.sum(normalize))
    return normalize_confidence

In [221]:
frequent_itemset, car, car_confidence = car_set(train_data, min_support, min_confidence, class_label)

In [222]:
rule_class, split_confidence, label = split_rule(car, car_confidence)

In [298]:
rule_class

{'Yes': [['FALSE'],
  ['Humidity=All'],
  ['Overcast'],
  ['Rainy'],
  ['Sunny'],
  ['TRUE'],
  ['Temperature=All'],
  ['Humidity=All', 'FALSE'],
  ['Overcast', 'FALSE'],
  ['FALSE', 'Rainy'],
  ['FALSE', 'Sunny'],
  ['FALSE', 'Temperature=All'],
  ['Humidity=All', 'Overcast'],
  ['Humidity=All', 'Rainy'],
  ['Humidity=All', 'Sunny'],
  ['Humidity=All', 'TRUE'],
  ['Humidity=All', 'Temperature=All'],
  ['Overcast', 'TRUE'],
  ['Overcast', 'Temperature=All'],
  ['Temperature=All', 'Rainy'],
  ['Temperature=All', 'Sunny'],
  ['Temperature=All', 'TRUE'],
  ['Humidity=All', 'Overcast', 'FALSE'],
  ['Humidity=All', 'FALSE', 'Rainy'],
  ['Humidity=All', 'FALSE', 'Sunny'],
  ['Humidity=All', 'FALSE', 'Temperature=All'],
  ['Overcast', 'FALSE', 'Temperature=All'],
  ['FALSE', 'Temperature=All', 'Rainy'],
  ['FALSE', 'Temperature=All', 'Sunny'],
  ['Humidity=All', 'Overcast', 'TRUE'],
  ['Humidity=All', 'Overcast', 'Temperature=All'],
  ['Humidity=All', 'Temperature=All', 'Rainy'],
  ['Humidity

In [240]:
normalize_confidence= normalize_confidence(split_confidence, label)

0.9999999999999998
1.0


In [226]:
BetP = []
for i in range(len(rule_class)): # 각 class에 대해
    # dictionary 형태로 BetP 생성(각 frequent itemset들에 대해서)
    dic_BetP = {}
    for p in frequent_itemset:
        dic_BetP[p] = 0
    
    for j in range(len(rule_class[label[i]])): # 각 데이터에 대해
        for k in range(len(rule_class[label[i]][j])): # 각 feature에 대해
            dic_BetP[rule_class[label[i]][j][k]] = dic_BetP[rule_class[label[i]][j][k]] + (normalize_confidence[i][j]/len(rule_class[label[i]][j]))
    BetP.append(dic_BetP)

In [297]:
BetP

[{'FALSE': 0.16862976050255202,
  'Humidity=All': 0.19167648213584607,
  'Overcast': 0.14063604240282682,
  'Rainy': 0.07597173144876325,
  'Sunny': 0.13506085590891242,
  'TRUE': 0.10031409501374167,
  'Temperature=All': 0.18771103258735763},
 {'FALSE': 0.1324881141045959,
  'Humidity=All': 0.22810353935552044,
  'Overcast': 0,
  'Rainy': 0.14791336502905442,
  'Sunny': 0.14104595879556261,
  'TRUE': 0.14458531431590071,
  'Temperature=All': 0.20586370839936616}]

## Predict

In [262]:
frequent_itemset, test_data, test_data[0][0]

(['FALSE',
  'Humidity=All',
  'Overcast',
  'Rainy',
  'Sunny',
  'TRUE',
  'Temperature=All'],
 [['Rainy', 'Humidity=All', 'Temperature=All', 'FALSE', 'Yes'],
  ['Overcast', 'Humidity=All', 'Temperature=All', 'TRUE', 'Yes'],
  ['Rainy', 'Humidity=All', 'Temperature=All', 'FALSE', 'Yes'],
  ['Sunny', 'Humidity=All', 'Temperature=All', 'TRUE', 'Yes'],
  ['Rainy', 'Humidity=All', 'Temperature=All', 'TRUE', 'No']],
 'Rainy')

In [275]:
binary_test = np.zeros([len(test_data),len(frequent_itemset)])
for i in range(len(test_data)):
    for j in range(len(test_data[i])):
        for frequent_item in frequent_itemset:
            if test_data[i][j] == frequent_item:
                binary_test[i][frequent_itemset.index(test_data[i][j])] +=1
                continue

In [280]:
BetP_list = []
for i in BetP:
    BetP_list.append(list(i.values()))
BetP_array = np.transpose(np.array(BetP_list))

In [287]:
result = np.matmul(binary_test, BetP_array)
predict = []
for i in range(len(result)):
    argmax = np.argmax(result[i])
    predict.append(label[argmax])
predict

['No', 'Yes', 'No', 'No', 'No']

In [288]:
'Yes', 'Yes', 'Yes', 'Yes', 'No'

array([[0.62398901, 0.71436873],
       [0.62033765, 0.57855256],
       [0.62398901, 0.71436873],
       [0.61476247, 0.71959852],
       [0.55567334, 0.72646593]])